In [3]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

#, store={}, item_uuids=[]
env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 



set_to_release = ['1ec54a03-0aaa-4d17-91d5-7a6ae3b12157']


store, uuids = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store_frame='object')
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1))

669
669
23.765299081802368


In [4]:
# TODO
# Check audits

# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = False
delete_problematic = False

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = ['experiment_hi_c', 'experiment_mic', 'experiment_seq']
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(store['experiment_hi_c']), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
for a_file in store['file_processed']:
    if a_file['file_format'] == '/file-formats/pairs/':
        if not a_file.get('quality_metric'):
            print(a_file['accession'], 'missing Pairsqc')
    if not a_file.get('source_experiments'):
        print(a_file['accession'], 'missing source experiments')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
for wfr in store['workflow_run_awsem']:
    if wfr['run_status'] != 'complete':
        print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i,x['uuid'],x['status']] for x in store[i] if x['status'] in ['deleted', 'replaced', 'archived', 'revoked']]
    if weird:
        for case in weird:
            print(case)
        print()


EXPSET CHECK
4DNESIO87EN4 missing processing tag ChIP-seq of untreated CTCF-AID mouse embryonic ste
4DNESX6IF3FC missing processing tag RNA-seq of  CTCF-AID mouse embryonic stem cells tr
4DNESD7LFIDK missing processing tag RNA-seq of  CTCF-AID mouse embryonic stem cells tr
4DNESO2RRVT8 missing processing tag ChIP-seq of  CTCF-AID washed off mouse embryonic s
4DNESKIB4QKT missing processing tag ChIP-seq of CTCF-AID mouse embryonic stem cells tr
4DNESLFS4XDP missing processing tag RNA-seq of CTCF-AID mouse embryonic stem cells tre
4DNESR2648ET missing processing tag RNA-seq of washed off CTCF-AID mouse embryonic ste
4DNES49QC878 missing processing tag ChIP-seq of  CTCF-AID mouse embryonic stem cells t
4DNESKPSB8I2 missing processing tag RNA-seq of untreated CTCF-AID mouse embryonic stem
4DNESJYLRY6V missing processing tag RNA-seq of untagged mouse embryonic stem cells tre
4DNESLTPD5EZ missing processing tag ChIP-seq of Input for CTCF of CTCF-AID mouse embry
4DNES6VO5VSG missing processin

In [5]:
# Check status
change_status = 'released'

common_types = ['lab', 'user', 'award','ontology', 'ontology_term', 
                'file_format', 'software', 'workflow', 'static_section',
                'organism', 'vendor', 'file_reference', 'individual_human', 
                'enzyme', 'biosource', 'file_format']

print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
print('---------------------------------\n')

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] != change_status:
            print(a_type, raw_data['uuid'], raw_data['status'])



COMMON ITEMS
COMMON ITEM NOT RELEASED static_section released to project 7627f4eb-9f2d-4171-9e9b-87ab800ab5cd
---------------------------------

publication 1ec54a03-0aaa-4d17-91d5-7a6ae3b12157 current
file_processed fddc0fa0-e44d-4a73-9ce7-d62b8a3f8433 uploaded
file_processed da99f429-20ab-4d91-bbfe-f8bdea8ed7b1 uploaded
file_processed b9a934b6-ba58-4c64-b559-1c72422a82c5 uploaded
file_processed ab57c9b8-e288-4ac6-b915-b0749766e736 uploaded
file_processed a9bce30d-e6cf-44b7-b8dc-bd04dde1932f uploaded
file_processed 3b632097-b378-44a1-b2c5-4525bb8d1195 uploaded
file_processed 3998924b-b533-4dce-888e-81aaefa21ba2 uploaded
file_processed 243f48b0-6b59-4f37-ad39-a12e0986f530 uploaded
file_processed 0ca0f4e3-30ca-4332-b735-e32dce08e1a4 uploaded
file_processed f15c79c1-d29a-4fda-a485-a6a5d2619ff4 uploaded
file_processed daae6f2d-fd1f-4624-aade-8cb40aa5b386 uploaded
file_processed af39847e-596a-4f82-9a48-771c41d2d0b9 uploaded
file_processed 69a74060-45fa-439b-883e-dd23f0d11669 uploaded
file_

In [6]:
# Release the items
action = True
change_status = 'released'

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': change_status}
            if action:
                if change_status == 'released':
                    if a_type in ['publication', 'user', 'award', 'vendor', 'organism', ]:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                else:
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)

ALREADY RELEASED/TO PROJECT publication current 1ec54a03-0aaa-4d17-91d5-7a6ae3b12157
file_processed uploaded fddc0fa0-e44d-4a73-9ce7-d62b8a3f8433
file_processed uploaded da99f429-20ab-4d91-bbfe-f8bdea8ed7b1
file_processed uploaded b9a934b6-ba58-4c64-b559-1c72422a82c5
file_processed uploaded ab57c9b8-e288-4ac6-b915-b0749766e736
file_processed uploaded a9bce30d-e6cf-44b7-b8dc-bd04dde1932f
file_processed uploaded 3b632097-b378-44a1-b2c5-4525bb8d1195
file_processed uploaded 3998924b-b533-4dce-888e-81aaefa21ba2
file_processed uploaded 243f48b0-6b59-4f37-ad39-a12e0986f530
file_processed uploaded 0ca0f4e3-30ca-4332-b735-e32dce08e1a4
file_processed uploaded f15c79c1-d29a-4fda-a485-a6a5d2619ff4
file_processed uploaded daae6f2d-fd1f-4624-aade-8cb40aa5b386
file_processed uploaded af39847e-596a-4f82-9a48-771c41d2d0b9
file_processed uploaded 69a74060-45fa-439b-883e-dd23f0d11669
file_processed uploaded 5869c455-30f9-44af-b948-8b7ce03684c6
file_processed uploaded 0375db9c-5b1a-4df8-a504-009242c2d6c0


workflow_run_awsem in review by lab b9ab4638-d6c7-4005-8fea-379cbd8d111a
workflow_run_awsem in review by lab af9e7ab9-8276-4225-b46f-0cd7c888df71
workflow_run_awsem in review by lab aea58e1a-7f18-430f-9710-d559fed40190
workflow_run_awsem in review by lab ad03d72d-ae7c-4eee-b87e-8feba21cdc17
workflow_run_awsem in review by lab a8b87649-35ab-4958-a157-d7f131a78e91
workflow_run_awsem in review by lab a527d4b4-2314-4459-aec4-e44d5f6382c3
workflow_run_awsem in review by lab a4f38c59-f4c6-4161-bcda-bee889f98383
workflow_run_awsem in review by lab a3b1fe8e-4e10-4ae5-93a5-20cf1045b3ec
workflow_run_awsem in review by lab a29817af-f0bb-4bb0-8bba-0852cec23250
workflow_run_awsem in review by lab 9dd81f1b-23c9-43f9-83a5-be2a884a1c18
workflow_run_awsem in review by lab 9cdc1fe4-8e30-43c6-9e4c-e9053bd50951
workflow_run_awsem in review by lab 896c7a46-420c-4e5a-88ed-a38278c9e890
workflow_run_awsem in review by lab 86384881-6f2d-4c48-9a3f-61d60c92f2e2
workflow_run_awsem in review by lab 85b3cd27-be11-4

workflow_run_awsem in review by lab e85866ff-060b-4107-baca-2d6a0370a3a4
workflow_run_awsem in review by lab e58622c8-cefd-4f2f-a13a-1c55524143f3
workflow_run_awsem in review by lab d66337da-ad46-4ea0-b39e-426d7757688a
workflow_run_awsem in review by lab d26a1c65-04b7-4165-9f74-75daa149c3ac
workflow_run_awsem in review by lab c7ac8883-2b21-406e-9503-3b0530409130
workflow_run_awsem in review by lab c673d248-9e9e-464b-8fd7-effc5dd005d4
workflow_run_awsem in review by lab c4d83566-bf86-4f0d-af9b-0757ffb2765f
workflow_run_awsem in review by lab c098ac44-8869-4b3d-8d81-9e216310fb95
workflow_run_awsem in review by lab bc8a9f21-06ed-4daf-8220-5044dc138f44
workflow_run_awsem in review by lab bc46b782-a5ce-4d55-8931-823850188675
workflow_run_awsem in review by lab bb1c7ea5-4b68-42a4-ae8a-801b274a684a
workflow_run_awsem in review by lab a9eb267a-cc86-47c8-b65f-f2e38d2a487c
workflow_run_awsem in review by lab a70204e6-b954-43d9-9e86-a604c97bb79b
workflow_run_awsem in review by lab a6e2d00b-904e-4